In [44]:
import mysql.connector
import pandas as pd
import pandas.api.types as pdtypes
import os

# WP Database credentials

In [45]:
wp_db = "linter_wp"
username = "root"
password = "*tH5WHzhH9.w"
host = "localhost"

# Create connection objects

In [46]:
wp_connection = mysql.connector.connect(
    database=wp_db, user=username, password=password, host=host
)

# Create cursor objects to execute queries

In [47]:
wp_cursor = wp_connection.cursor()

# Function to convert table to dataframe

In [48]:
def get_table_data(cursor, table_name):
  query = f"SELECT * FROM {table_name}"
  cursor.execute(query)
  data = cursor.fetchall()
  column_names = [i[0] for i in cursor.description]  # Get column names
  df = pd.DataFrame(data, columns=column_names)  # Create DataFrame
  return df

# WP - Users Table

In [49]:
users_wp_df = get_table_data(wp_cursor,"wp_users") 
users_wp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13781 entries, 0 to 13780
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   13781 non-null  int64         
 1   user_login           13781 non-null  object        
 2   user_pass            13781 non-null  object        
 3   user_nicename        13781 non-null  object        
 4   user_email           13781 non-null  object        
 5   user_url             13781 non-null  object        
 6   user_registered      13781 non-null  datetime64[ns]
 7   user_activation_key  13781 non-null  object        
 8   user_status          13781 non-null  int64         
 9   display_name         13781 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 1.1+ MB


# WP - Usermeta Table

In [50]:
usermeta_wp_df = get_table_data(wp_cursor,"wp_usermeta")
usermeta_wp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58063 entries, 0 to 58062
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   umeta_id    58063 non-null  int64 
 1   user_id     58063 non-null  int64 
 2   meta_key    58063 non-null  object
 3   meta_value  58048 non-null  object
dtypes: int64(2), object(2)
memory usage: 1.8+ MB


# Merged Table

In [51]:
merged_users_df = users_wp_df.merge(usermeta_wp_df, left_on='ID', right_on='user_id', how='inner')
merged_users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58063 entries, 0 to 58062
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   58063 non-null  int64         
 1   user_login           58063 non-null  object        
 2   user_pass            58063 non-null  object        
 3   user_nicename        58063 non-null  object        
 4   user_email           58063 non-null  object        
 5   user_url             58063 non-null  object        
 6   user_registered      58063 non-null  datetime64[ns]
 7   user_activation_key  58063 non-null  object        
 8   user_status          58063 non-null  int64         
 9   display_name         58063 non-null  object        
 10  umeta_id             58063 non-null  int64         
 11  user_id              58063 non-null  int64         
 12  meta_key             58063 non-null  object        
 13  meta_value           58048 non-

## Meta_key list

In [52]:
# Get a list of unique values from the 'meta_key' column
unique_meta_keys = merged_users_df['meta_key'].unique()
unique_meta_keys

array(['nickname', 'first_name', 'last_name', 'description',
       'rich_editing', 'syntax_highlighting', 'comment_shortcuts',
       'admin_color', 'use_ssl', 'show_admin_bar_front', 'locale',
       'wp_capabilities', 'wp_user_level', 'dismissed_wp_pointers',
       'show_welcome_panel', 'wp_user-settings', 'wp_user-settings-time',
       'wp_dashboard_quick_press_last_post_id',
       'community-events-location', 'closedpostboxes_dashboard',
       'metaboxhidden_dashboard', 'session_tokens', 'online_status',
       'elementor_introduction', 'elementor_admin_notices',
       'elementor_preferences', 'wp_persisted_preferences',
       'nav_menu_recently_edited', 'managenav-menuscolumnshidden',
       'metaboxhidden_nav-menus', 'wp_elementor_connect_common_data',
       'meta-box-order_dashboard', 'wpcf7_hide_welcome_panel_on',
       'closedpostboxes_rtcl_cfg', 'metaboxhidden_rtcl_cfg',
       'edit_page_per_page', '_rtcl_ads', 'thechamp_avatar',
       'thechamp_large_avatar', 'the

# Setup the mailchimp dataframe

In [53]:
# Create the mailchimp dataframe
mailchimp_df = pd.DataFrame({'email_address': users_wp_df['user_email']})
mailchimp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13781 entries, 0 to 13780
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   email_address  13781 non-null  object
dtypes: object(1)
memory usage: 107.8+ KB


## create the detailed dataframes

In [54]:
# Filter for rows where 'meta_key' is 'first_name'
first_name_df = merged_users_df[merged_users_df['meta_key'] == 'first_name']
merged_tmp_df = pd.merge(users_wp_df, first_name_df, left_on='ID', right_on='user_id', how='inner')
mailchimp_df['FNAME'] = merged_tmp_df['meta_value']

# Filter for rows where 'meta_key' is 'last_name'
last_name_df = merged_users_df[merged_users_df['meta_key'] == 'last_name']
merged_tmp_df = pd.merge(users_wp_df, last_name_df, left_on='ID', right_on='user_id', how='inner')
mailchimp_df['LNAME'] = merged_tmp_df['meta_value']

mailchimp_df['merge_fields'] = mailchimp_df.apply(lambda row: 
                                                 {'FNAME': row['FNAME'], 'LNAME': row['LNAME']}, 
                                                 axis=1)

# Filter for rows where 'meta_key' is '_rtcl_geo_address'
address_df = merged_users_df[merged_users_df['meta_key'] == '_rtcl_latitude']
merged_tmp_df = pd.merge(users_wp_df, address_df, left_on='ID', right_on='user_id', how='inner')
mailchimp_df['Address'] = merged_tmp_df['meta_value']

# Filter for rows where 'meta_key' is '_rtcl_phone'
phone_df = merged_users_df[merged_users_df['meta_key'] == '_rtcl_phone']
merged_tmp_df = pd.merge(users_wp_df, phone_df, left_on='ID', right_on='user_id', how='inner')
mailchimp_df['Phone Number'] = merged_tmp_df['meta_value']

mailchimp_df['Tags'] = 'Customer'
mailchimp_df['status'] = 'subscribed'

mailchimp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13781 entries, 0 to 13780
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   email_address  13781 non-null  object
 1   FNAME          1017 non-null   object
 2   LNAME          1017 non-null   object
 3   merge_fields   13781 non-null  object
 4   Address        0 non-null      object
 5   Phone Number   13757 non-null  object
 6   Tags           13781 non-null  object
 7   status         13781 non-null  object
dtypes: object(8)
memory usage: 861.4+ KB


# Export the dataframe to Mailchimp

In [55]:
# Export the DataFrame to a CSV file
csv_file_path = "./Data/Contacts.csv"
mailchimp_df.to_csv(csv_file_path, index=False)

print(f"WordPress users exported to {csv_file_path}")

WordPress users exported to ./Data/Contacts.csv


In [56]:
# Save the DataFrame to a JSON file
json_file_path = './Data/Contacts.json'
mailchimp_df.to_json(json_file_path, orient='index', indent=4) 
print(f"WordPress users exported to {json_file_path}")

WordPress users exported to ./Data/Contacts.json
